In [2]:
import sys,re
import os
from pathlib import Path

project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(project_root))

from core.backtesting_opt import _Data, Strategy, Backtest
from utils.functions import BlackScholes as bs
import pandas as pd
import numpy as np
from collections import deque
import math
from scipy.optimize import root_scalar
from datetime import datetime, date as DateObject,time



In [4]:
class IV_Slope(Strategy):
    upper_threshold = 0.19
    lower_threshold = 0.15
    position_id= 0
    
    def init(self):
        self.legs = {
            'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg2': {'type': 'PE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg3': {'type': 'CE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg4': {'type': 'PE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None}
        }
        # Flag to track if we've already executed today
        self.executed_today = False
        self.last_execution_date = None
        self.previous_signal = 0
        self.lowiv_w=0
        self.highiv_w=0
        self.lowiv_m=0
        self.highiv_m=0
        self.start=0
        self.start_date=None
        self.current_m=0
        self.current_w=0
        self.iv_rank=0
        self.in_pos=0
        self.seventy_th=False 
        self.twenty_th=False
        # self.iv_data=deque()
    
    def execution_window(self, start_time_str="15:00:00", end_time_str="15:29:00"):
        current_time = pd.Timestamp(self.time).time()
        current_date = pd.Timestamp(self.time).date()
        
        # Reset execution flag for new trading day
        if self.last_execution_date != current_date:
            self.executed_today = False
            self.last_execution_date = current_date
        
        # Check if current time is within execution window
        start_time = pd.Timestamp(start_time_str).time()
        end_time = pd.Timestamp(end_time_str).time()
        
        if not (start_time <= current_time <= end_time):
            return False
            
        # Execute only once per day during the time window
        if self.executed_today:
            return False
            
        # Mark as executed for today
        self.executed_today = True
        return True
    
    def extract_strike(self,contract: str) -> int:
        match = re.search(r'(\d+)(CE|PE)$', contract)
        if match:
            return int(match.group(1))
        else:
            raise ValueError(f"Could not extract strike from: {contract}")
    
    def next(self):
        current_time=pd.Timestamp(self.time).time()
        current_date = pd.Timestamp(self.time).date()
        # print(f"TODAY'S TIME IS : {self.time}")
        if self.spot is None or pd.isna(self.spot):
            return
            
        # Check if we should execute today
        # if not self.execution_window():
        #     return
        atm = round(self.spot / 50) * 50
  
        for leg in self.legs.values():
            available_ttes = list(self.tte_to_expiry.keys()) if self.tte_to_expiry else []
            lower, upper = leg["expiry_range"]
            matching_ttes = [tte for tte in available_ttes if lower <= tte <= upper]
            if not matching_ttes:
                print(f"No matching TTEs found for leg {leg['type']} with range {leg['expiry_range']}. Available TTEs: {sorted(available_ttes) if available_ttes else 'None'}")
                return 
            
            valid_tte = min(matching_ttes)
            leg["expiry"] = self.tte_to_expiry[valid_tte]
            
            leg["strike"] = float(atm)
            contract = f"NIFTY{pd.Timestamp(leg['expiry']).strftime('%d%b%y').upper()}{int(leg['strike'])}{leg['type']}"
            leg["contract"] = contract
            leg["data"] = self._data.get_ticker_data(contract)
        # print(self.legs)

        missing_legs = [leg["contract"] for leg in self.legs.values() if leg["data"] is None]
        if missing_legs:
            print(f"IV not found for {self.time}. Spot: {self.spot} Missing legs: {missing_legs}")
            return
        
        if current_time==time(15,29):
            if self.start==0:
                ivce=self.legs["leg1"]['data']['iv']
                ivpe=self.legs["leg2"]['data']['iv']
                iv_avg=(ivce+ivpe)/2
                print(iv_avg)
                self.current_w=iv_avg
                self.lowiv_w=iv_avg
                self.highiv_w=iv_avg

                ivce=self.legs["leg3"]['data']['iv']
                ivpe=self.legs["leg4"]['data']['iv']
                iv_avg=(ivce+ivpe)/2
                print(iv_avg)
                self.current_m=iv_avg
                self.lowiv_m=iv_avg
                self.highiv_m=iv_avg

                self.start+=1
                self.start_date=current_date
                # self.iv_data.append((self.lowiv_w,self.highiv_w,self.lowiv_m,self.highiv_m))

            else:
                print(f"lowivw: {self.lowiv_w} highivw: {self.highiv_w} lowivm: {self.lowiv_m} highivm: {self.highiv_m}")
            
                ivce=self.legs["leg1"]['data']['iv']
                ivpe=self.legs["leg2"]['data']['iv']
                iv_avg=(ivce+ivpe)/2
                print(iv_avg)
                self.current_w=iv_avg
                if self.lowiv_w > iv_avg:
                    self.lowiv_w=iv_avg
                if self.highiv_w < iv_avg:
                    self.highiv_w=iv_avg

                ivce=self.legs["leg3"]['data']['iv']
                ivpe=self.legs["leg4"]['data']['iv']
                iv_avg=(ivce+ivpe)/2
                print(iv_avg)
                self.current_m=iv_avg
                if self.lowiv_m > iv_avg:
                    self.lowiv_m=iv_avg
                if self.highiv_m < iv_avg:
                    self.highiv_m=iv_avg
                # self.iv_data.append((self.lowiv_w,self.highiv_w,self.lowiv_m,self.highiv_m))
                print(f"lowivw: {self.lowiv_w} highivw: {self.highiv_w} lowivm: {self.lowiv_m} highivm: {self.highiv_m}")
                

        if self.start_date!=None and (current_date-self.start_date).days >365:
            print("Year ended!")
            self.iv_rank=((self.current_w-self.lowiv_w)/(self.highiv_w-self.lowiv_w))*100
            print(f"IV RANK OF WEEKLY IS : {self.iv_rank:.5f}")
        
        # iv_slope = math.log((self.legs["leg1"]["data"]["iv"] + self.legs["leg2"]["data"]["iv"]) / (self.legs["leg3"]["data"]["iv"] + self.legs["leg4"]["data"]["iv"]), 10)
        if self.in_pos==0 and self.iv_rank!=0 and self.iv_rank>70:
            #sell atm ce pe weekly and buy atm ce pe monthly 
         
            self.sell(strategy_id='strativrank',position_id=self.position_id,leg_id=1,ticker=self.legs['leg1']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="short")
            self.sell(strategy_id='strativrank',position_id=self.position_id,leg_id=2,ticker=self.legs['leg2']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="short")
            self.buy(strategy_id='strativrank',position_id=self.position_id,leg_id=3,ticker=self.legs['leg3']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="longpe")
            self.buy(strategy_id='strativrank',position_id=self.position_id,leg_id=4,ticker=self.legs['leg4']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="longpe")
            self.in_pos+=1
            self.seventy_th=True

        if self.in_pos==0 and self.iv_rank!=0 and self.iv_rank<20:
            #sell atm ce pe weekly and buy atm ce pe monthly 
            self.sell(strategy_id='strativrank',position_id=self.position_id,leg_id=1,ticker=self.legs['leg1']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="short")
            self.sell(strategy_id='strativrank',position_id=self.position_id,leg_id=2,ticker=self.legs['leg2']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="short")
            self.buy(strategy_id='strativrank',position_id=self.position_id,leg_id=3,ticker=self.legs['leg3']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="longpe")
            self.buy(strategy_id='strativrank',position_id=self.position_id,leg_id=4,ticker=self.legs['leg4']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="longpe") 
            self.in_pos+=1
            self.twenty_th=True 
        
        #checking breakeven adjustment!
        if self.seventy_th and self.current_w < 70:
            for trade in self.active_trades:
                trade.close(trade.size,"closing_due_to_near_expiry_match") 
            
            self.seventy_th=False

        elif self.twenty_th and self.current_w > 20:
            for trade in self.active_trades:
                trade.close(trade.size,"closing_due_to_near_expiry_match") 
            
            self.twenty_th=False

        for trade in self.active_trades:
            entry_price=trade.entry_price
            cont=trade.ticker

            entry_strike=self.extract_strike(cont)
            if entry_strike and trade.entry_tag=='short' and atm>=(entry_strike+entry_price) and self.in_pos==1:
                #if we are in 1 position add another calender too.with srike of +100 from this point
                self.sell(strategy_id='strativrank',position_id=self.position_id,leg_id=1,ticker=self.legs['leg1']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="short")
                self.sell(strategy_id='strativrank',position_id=self.position_id,leg_id=2,ticker=self.legs['leg2']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="short")
                self.buy(strategy_id='strativrank',position_id=self.position_id,leg_id=3,ticker=self.legs['leg3']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="longpe")
                self.buy(strategy_id='strativrank',position_id=self.position_id,leg_id=4,ticker=self.legs['leg4']['contract'],quantity=1,stop_loss=0.0,take_profit=0.0,tag="longpe")
                self.in_pos+=1
                



        near_expiry = None
        for trade in self.active_trades:
            expiry = datetime.strptime(trade.ticker[-14:-7], "%d%b%y").date()
            near_expiry = expiry if near_expiry is None else min(near_expiry, expiry)

        
            if current_date==near_expiry:
                trade.close(trade.size,"closing_due_to_near_expiry_match")




        # active_trades = self.active_trades
        # new_signal = 0
        # if iv_slope > self.upper_threshold:
        #     new_signal = -1
        # elif iv_slope < self.lower_threshold:
        #     new_signal = 1
        
        # if new_signal == -1:
        #     self.entry_type_dict = {'weekly': 'SELL', 'monthly': 'BUY'}
        # elif new_signal == 1:
        #     self.entry_type_dict = {'weekly': 'BUY', 'monthly': 'SELL'}
        # elif new_signal == 0:
        #     return
            
        
        # if not active_trades:
            
        #     placed_any_leg = False
        #     for leg_id, leg in self.legs.items():
        #         entry_type = self.entry_type_dict.get(leg["expiry_type"])
        #         order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
        #         if order_fn is None:
        #             continue
        #         order_fn(
        #             strategy_id='strat1',
        #             position_id=self.position_id,
        #             leg_id=leg_id,
        #             ticker=leg["contract"],
        #             quantity=1,
        #             stop_loss=None,
        #             take_profit=None,
        #             tag=f'{new_signal} signal entry'
        #         )
        #         placed_any_leg = True
        #     if placed_any_leg:
        #         self.position_id += 1
        #         self.previous_signal = new_signal
        # else:
        #     near_expiry = None
        #     for trade in active_trades:
        #         expiry = datetime.strptime(trade.ticker[-14:-7], "%d%b%y").date()
        #         near_expiry = expiry if near_expiry is None else min(near_expiry, expiry)
            
        #     exit_reason = (
        #         "Near Expiry reached" if (pd.Timestamp(self.time).date() == near_expiry) else
        #         "Signal changed" if (self.previous_signal != new_signal) else
        #         None
        #     )
            
        #     if exit_reason:
        #         for trade in active_trades:
        #             print("Closing position")
        #             trade.close(trade.size, tag=exit_reason)
        #         self.previous_signal = 0
                
        #         placed_any_leg = False
        #         for leg_id, leg in self.legs.items():
        #             entry_type = self.entry_type_dict.get(leg["expiry_type"])
        #             order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
        #             if order_fn is None:
        #                 continue
        #             order_fn(
        #                 strategy_id='strat1',
        #                 position_id=self.position_id,
        #                 leg_id=leg_id,
        #                 ticker=leg["contract"],
        #                 quantity=1,
        #                 stop_loss=None,
        #                 take_profit=None,
        #                 tag=f'{new_signal} signal entry'
        #             )
        #             placed_any_leg = True
        #         if placed_any_leg:
        #             self.position_id += 1
        #             self.previous_signal = new_signal
                
                
        #     else:
        #         if self.previous_signal == new_signal:
        #             leg_strike = self.legs["leg4"]["strike"]
        #             if (0.99*leg_strike) <= self.spot <= (1.01*leg_strike):
        #                 pass
        #             else:
                            
        #                     # Place orders according to entry type
        #                     placed_any_leg = False
        #                     for leg_id, leg in self.legs.items():
        #                         entry_type = self.entry_type_dict.get(leg["expiry_type"])
        #                         order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
        #                         if order_fn is None:
        #                             continue
        #                         print(f"Placing {entry_type} order for {leg_id}: {leg['contract']}")
        #                         order_fn(
        #                             strategy_id='strat1',
        #                             position_id=self.position_id,
        #                             leg_id=leg_id,
        #                             ticker=leg["contract"],
        #                             quantity=1,
        #                             stop_loss=None,
        #                             take_profit=None,
        #                             tag=f'Adjustment {entry_type} at ATM {atm}'
        #                         )
        #                         placed_any_leg = True
                            
        #                     if placed_any_leg:
        #                         self.position_id += 1

In [5]:
db_path = "nifty_1min_desiquant_new.duckdb"
# db_path = "nifty_opt_icici_1min_new2.duckdb"

bt = Backtest(db_path=db_path, strategy=IV_Slope, cash=10000000, commission_per_contract=0.65, option_multiplier=75)
stats = bt.run(start_date="2021-01-01", end_date="2022-01-31")
print(stats)

Identifying tables in date range...
Found 267 tables out of 815 total tables with data in range


Backtesting Options Strategy:   0%|          | 0/267 [00:00<?, ?it/s]

IV not found for 2021-01-01 09:15:00. Spot: 14014.849609375 Missing legs: ['NIFTY21JAN2114000CE', 'NIFTY21JAN2114000PE']
IV not found for 2021-01-01 09:30:00. Spot: 14026.2998046875 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 09:31:00. Spot: 14026.400390625 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 09:44:00. Spot: 14025.0498046875 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 09:45:00. Spot: 14030.900390625 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 09:46:00. Spot: 14031.400390625 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 09:47:00. Spot: 14030.75 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 09:48:00. Spot: 14031.2001953125 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 09:49:00. Spot: 14030.9501953125 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 09:50:00. Spot: 14028.25 Missing legs: ['NIFTY21JAN2114050PE']
IV not 

Backtesting Options Strategy:   0%|          | 1/267 [00:00<00:43,  6.16it/s]

IV not found for 2021-01-01 13:13:00. Spot: 14032.650390625 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:14:00. Spot: 14030.349609375 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:15:00. Spot: 14030.7001953125 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:16:00. Spot: 14031.7998046875 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:17:00. Spot: 14034.75 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:18:00. Spot: 14032.7998046875 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:19:00. Spot: 14033.599609375 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:20:00. Spot: 14032.75 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:21:00. Spot: 14031.849609375 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:22:00. Spot: 14030.099609375 Missing legs: ['NIFTY21JAN2114050PE']
IV not found for 2021-01-01 13:

...

Backtesting Options Strategy:  99%|█████████▉| 264/267 [00:47<00:00,  4.61it/s]

Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS : 16.89329
Year ended!
IV RANK OF WEEKLY IS

Backtesting Options Strategy: 100%|█████████▉| 266/267 [00:48<00:00,  5.17it/s]

Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS : 26.08963
Year ended!
IV RANK OF WEEKLY IS

Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS : 24.54979
Year ended!
IV RANK OF WEEKLY IS

In [5]:
from core.wfo import WalkForwardOptimizer

constraint = lambda x: x['upper_threshold'] > x['lower_threshold']


wfo = WalkForwardOptimizer(
    strategy=IV_Slope,
    optimization_params={
        # 'upper_threshold': [-0.15 ,-0.1, 0.1, 0.15, 0.2, 0.25, 0.3],
        # 'lower_threshold': [-0.15 ,-0.1 ,0, 0.05, 0.1, 0.15, 0.2]
        'upper_threshold': [0.1, 0.15, 0.2, 0.25],
        'lower_threshold': [ 0.1, 0.15, 0.2]
    },
    # constraint=constraint,
    maximize='Sharpe Ratio',
)

Successfully imported local quantstats from: /Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/quantstats
Successfully imported plotting module from local quantstats


In [8]:
# def optimize_stock(self, stock, timeframe, db_path, exchange = None, training_candles = 2000, testing_candles = 200):
    
wfo.optimize_stock(
    stock='NIFTY',
    timeframe='1min',
    db_path='../core/nifty_opt_icici_1min.duckdb',
    exchange='NSE',
    training_candles=6000,
    testing_candles=3000,
    start_date='2022-01-01',
    end_date='2024-01-01',
    # method='sambo',
    # max_tries=10
)

Using date range: 2022-01-01 to 2024-01-01


Walk-Forward Optimizing NIFTY:   0%|          | 0/89 [00:00<?, ?it/s]

Identifying tables in date range...
Found 0 tables out of 0 total tables with data in range


In [9]:
bt.tear_sheet()

Created default benchmark with 467 data points
Filtered trade_stats_df:                                      Value
Statistic                                 
Start                  2022-01-03 00:00:00
End                    2023-11-22 00:00:00
Duration                 688 days 00:00:00
Exposure Time [%]                      0.0
Equity Final [$]                10008317.0
Equity Peak [$]                 10036224.0
Return [%]                         0.08317
Return (Ann.) [%]                 0.044775
Volatility (Ann.) [%]             0.353185
Sharpe Ratio                      0.126776
Sortino Ratio                     0.188602
Calmar Ratio                      0.161026
Max. Drawdown [%]                -0.278063
# Trades                                 4
Win Rate [%]                           0.0
Profit Factor                          0.0
Expectancy [%]                  -44.802521
Kelly Criterion                        NaN
No backtest object provided. Skipping equity curve generation.
HTML 